In [1]:
# Setup
# Instalación de librerías necesarias
!pip install -q -U google-genai scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.3/713.3 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 24.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.47.0 which is incompatible.


In [4]:
# Módulos

from google import genai
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import userdata


# --- CONFIGURACIÓN API Key ---
# En un entorno real, usar os.environ.get("GOOGLE_API_KEY")
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEYL')

client = genai.Client(api_key=GOOGLE_API_KEY)

In [6]:
# Modelos disponibles en Gemini
print("List of models that support generateContent:\n")
for m in client.models.list():
    for action in m.supported_actions:
        if action == "generateContent":
            print(m.name)

print("List of models that support embedContent:\n")
for m in client.models.list():
    for action in m.supported_actions:
        if action == "embedContent":
            print(m.name)

print("\n✅ Librerías importadas y entorno configurado.")

List of models that support generateContent:

models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-flash-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1

# 🧠 Cortex Nivel 5: LLMs y RAG (Generación Aumentada)

Aquí conectamos a Cortex con **Google Gemini**. Pero la memoria de un LLM es estática (se entrenó hace meses). Para que sepa sobre datos privados o recientes, usamos **RAG (Retrieval Augmented Generation)**.

## Conceptos Clave

### 1. Embeddings (Vectores Semánticos)
Las computadoras no entienden palabras, entienden números.
Un "Embedding" convierte texto en una lista de números (vector).
* *Ejemplo:* "Perro" y "Caniche" tendrán vectores matemáticamente cercanos. "Perro" y "Tostadora" estarán lejos.

### 2. Búsqueda Semántica
En lugar de buscar por palabras clave exactas (Ctrl+F), buscamos por significado usando la **Similitud del Coseno**.

### 3. El Prompt del Agente
No solo le decimos "responde". Le damos una **Persona** ("Eres un experto académico") y **Herramientas** ("Usa el contexto provisto").

In [15]:
# --- 1. MOTOR DE EMBEDDINGS (Hugging Face o Gemini) ---
# Usaremos Gemini para simplificar, pero el concepto es igual con HF
def get_embedding(text):
    result = client.models.embed_content(
        model="models/text-embedding-004",
        contents=[text]) # contents expects a list of strings

    [embedding_obj] = result.embeddings

    return embedding_obj.values

# --- 2. MINI BASE VECTORIAL (Memoria RAG) ---
knowledge_base = [
    "La fecha del examen final de Python es el 15 de Diciembre.",
    "El profesor de Análisis de Datos se llama Leandro.",
    "Para aprobar se necesita un promedio mayor a 70."
]

# Convertimos texto a vectores
vector_db = [get_embedding(doc) for doc in knowledge_base]

# --- 3. BÚSQUEDA SEMÁNTICA ---
query = "¿Cuándo rindo y qué nota necesito?"
query_vector = get_embedding(query)

# Calculamos similitud (Matemática vectorial)
# Reshape necesario porque sklearn espera matrices
similarities = cosine_similarity(
    [query_vector],
    vector_db
)

# Obtenemos el índice del documento más similar
best_idx = np.argmax(similarities)
retrieved_info = knowledge_base[best_idx]

print(f"👤 Pregunta: {query}")
print(f"🤖 Contexto Recuperado (RAG): '{retrieved_info}'")

# --- 4. GENERACIÓN (LLM) ---

prompt = f"""
Eres un asistente universitario. Responde usando SOLO el siguiente contexto.
Contexto: {retrieved_info}
Pregunta: {query}
"""

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
)

print(f"💬 Respuesta Gemini: {response.text}")

👤 Pregunta: ¿Cuándo rindo y qué nota necesito?
🤖 Contexto Recuperado (RAG): 'Para aprobar se necesita un promedio mayor a 70.'
💬 Respuesta Gemini: Para aprobar se necesita un promedio mayor a 70.

No dispongo de información sobre cuándo rindes.
